# Regression algorithms

## Objective

The goal of this notebook is to manipulate some regression algorithms using Python. We will use [Pandas](https://pandas.pydata.org/), [Numpy](https://www.numpy.org/), and [scikit-learn](http://scikit-learn.org/stable/documentation.html).

## Dataset

We willl use the `Boston` dataset. It comprises housing values in suburbs of Boston.


This notebook was created using:
    * python 3.7.1
    * matplotlib 3.0.2
    * pandas 0.23.4
    * numpy 1.15.4
    * seaborn 0.9.0
    * scikit-learn 0.20.1

You can check your version of Python by running
```python
import sys
print(sys.version)
```

and the version of any module by running

```python
import <module name>
print(<module name>.__version__)
```

### Loading the libraries to manipulate the data

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

%matplotlib inline

## 1. Predicting Housing Price

Boston house prices dataset
---------------------------

* **Data Set Characteristics:**  

    * Attribute Information (in order):
        - `CRIM`:     per capita crime rate by town
        - `ZN`:       proportion of residential land zoned for lots over 25,000 sq.ft.
        - `INDUS`:    proportion of non-retail business acres per town
        - `CHAS`:     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - `NOX`:      nitric oxides concentration (parts per 10 million)
        - `RM`:       average number of rooms per dwelling
        - `AGE`:      proportion of owner-occupied units built prior to 1940
        - `DIS`:      weighted distances to five Boston employment centres
        - `RAD`:      index of accessibility to radial highways
        - `TAX`:      full-value property-tax rate per \$10,000
        - `PTRATIO`:  pupil-teacher ratio by town
        - `B`:        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - `LSTAT`:    % lower status of the population
        - `MEDV`:     Median value of owner-occupied homes in $1000's


The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
...', Wiley, 1980.   N.B. Various transformations are used in the table on pages 244-261 of the latter.

The Boston house-price data has been used in many machine learning papers that address regression
problems.   
     
References

   - Belsley, Kuh & Welsch, 'Regression diagnostics: Identifying Influential Data and Sources of Collinearity', Wiley, 1980. 244-261.
   - Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning. In Proceedings on the Tenth International Conference of Machine Learning, 236-243, University of Massachusetts, Amherst. Morgan Kaufmann.

### Importing the dataset

In [0]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data',\
                 header = None, sep = '\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',\
              'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df.head()

In [0]:
df.shape

### 2. Data preprocessing

Counting the number of missing values of each feature

In [0]:
df.isnull().sum()

# Exploratory Data Analysis (EDA)

In [0]:
sns.set(style = 'whitegrid', context = 'notebook')
cols = ['LSTAT', 'INDUS', 'NOX', 'RM', 'MEDV']
sns.pairplot(df[cols], height = 2.5)
plt.show()

There is a linear relationship between **RM** (the average number of rooms per dwelling) and the housing prices **MEDV** (Median value of owner-occupied homes in $1000's). **MEDV** seems to be normally distributed, but it contains outliers

### Checking the linear relationships between the variables through the correlation matrix

In [0]:
# sns.reset_orig()
sns.set(rc={'figure.figsize':(12,9)})
correlation_matrix = np.corrcoef(df[df.columns].values.T)
# sns.set(font_scale = 1.5)
hm = sns.heatmap(data = correlation_matrix,
                 annot = True,
                 square = True,
                 fmt='.2f',
                 yticklabels=df.columns,
                 xticklabels=df.columns)

* **MEDV** is positively high correlated with **RM**(0.7), whereas it has a strong negative correlation with **LSTAT**(-0.74)
* Features **RAD** and **Tax** are strongly correlated to each other (0.91). Thus, we should not select these feature together to train the model. The same are valid for features **NOX** and **DIS**, **AGE** and **DIS**, **INDUS** and **DIS**.

Let's investigate how **RM** and **LSTAT** vary with **MEDV**

In [0]:
plt.figure(figsize=(20,5))

features = ['RM', 'LSTAT']

for i, col in enumerate(features):
    plt.subplot(1, len(features), i + 1)
    plt.scatter(df[col], df['MEDV'])
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('MEDV')

* The price increases as the average number of rooms per dwelling (**RM**) increases. Therefore, there are some outliers
* The prices tends to decreases when **LSTAT** increases, although it does not seem to follows a linear relationship  

# Implementing an ordinary least squares (OLS)

## Estimating the parameters $\beta{}$ using sum of squared residuals (SSR), computed as:

$$RSS(\beta{}) = \min_{\beta{}\in{}\mathbb{R}} \sum_{i=1}^{N}{(y_i - x_i^T\beta{})^2}$$

In [0]:
class LinearRegressionGD(object):
    
    def __init__(self, alpha=0.001, n_iter=20):
        self.alpha = alpha
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.beta_ = np.zeros(1 + X.shape[1])
        self.cost_ = [] 
        
        for i in range(self.n_iter):
            output = self.net_input(X)
            errors = (y - output)
            self.beta_[1:] += self.alpha * X.T.dot(errors)
            self.beta_[0]  += self.alpha * errors.sum()
            cost = (errors ** 2).sum() / 2.0
            self.cost_.append(cost)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.beta_[1:]) + self.beta_[0]
    
    def predict(self, X):
        return self.net_input(X)

In [0]:
X = df[['RM']].values
y = df['MEDV'].values

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()

X_std = sc_x.fit_transform(X)
y_std = sc_y.fit_transform(y[:, np.newaxis]).flatten()

### Creating the model

In [0]:
lr_model = LinearRegressionGD()
lr_model.fit(X_std, y_std)

### Checking the cost as a function of the number of epochs

In [0]:
# sns.reset_orig()
plt.figure(figsize=(4,3))
plt.plot(range(1, lr_model.n_iter+1), lr_model.cost_)
plt.ylabel('SSE')
plt.xlabel('Epoch')
plt.tight_layout()

As can be seen, the gradient descent algorithm converged after the fifth epoch

### Checking how well the linear regression model fits the training data

In [0]:
def lin_regplot(X, y, model, xlabel='', ylabel=''):
    plt.scatter(X, y, c='blue')
    plt.plot(X, model.predict(X), color='red')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    return None

plt.figure(figsize=(5,3))
lin_regplot(X_std, y_std, lr_model, xlabel = 'RM (standardized)', ylabel = 'MEDV (standardized)')

* It seems that only the number of rooms is insufficient to explain the price of the houses in different cases.
* Likewise, the data suggest that prices may have been clipped. Thus, let's scale the predicted price outcome back on the price in \$1000's axis, using the **inverse_transform** method of the **_StandardScaler_**

In [0]:
num_rooms_std = sc_x.transform(np.array([[5.0]]))
# using the trained model to predict the price of a five-rooms house.
price_std = lr_model.predict(num_rooms_std) 
print("Price in $1000's: %.3f" % sc_y.inverse_transform(price_std))

In [0]:
print('Slope: %.3f' %lr_model.beta_[1])
print('Intercept: %.3f' %lr_model.beta_[0])

## Using scikit-learn to estimate the parameters

In [0]:
from sklearn.linear_model import LinearRegression

lmodel = LinearRegression()
lmodel.fit(X, y)

In [0]:
print('Slope: %.3f' %lmodel.coef_[0], '\nIntercept: %.3f' % lmodel.intercept_)

In [0]:
plt.figure(figsize=(5,3))
lin_regplot(X, y, lmodel, 'RM', 'MEDV')

## Using RANdom SAmple Consensus (RANSAC) algorithm to fit a robust regression model

## The RANSAC algorithm fits a regression model to a subset of data, aka _inliers_. 

It can be summarized as follows:

1. Select a random number of samples to be inliers and fit the model.
2. Test all other data points against the fitted model and those points that fall within a user-given tolerance to the inliers.
3. Refit the model using all the inliers.
4. Estimate the error of the fitted model versus the inliers.
5. Terminate if the performance meets a certain user-defined threshold of if a fixed number of iterations has been reached. Otherwise, returns to step 1.

In [0]:
from sklearn.linear_model import RANSACRegressor

ransac = RANSACRegressor(base_estimator=LinearRegression(), 
                         max_trials=100, 
                         min_samples=50, 
                         residual_threshold=5.0, 
                         random_state=10)
ransac.fit(X,y)

In [0]:
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
line_X = np.arange(3,10, 1)
line_y_ransac = ransac.predict(line_X[:, np.newaxis])

plt.figure(figsize=(5,3))
plt.scatter(X[inlier_mask], y[inlier_mask], c='blue', label = 'Inliers')
plt.scatter(X[outlier_mask], y[outlier_mask], c='green', label='Outliers', marker='s')
plt.plot(line_X, line_y_ransac, color='red')
plt.xlabel('RM')
plt.ylabel('MEDV')
plt.legend(loc='lower right')

print('Slope: %.3f' %ransac.estimator_.coef_[0], '\nIntercept: %.3f' %ransac.estimator_.intercept_)

# Evaluating the performance of linear models

## Splitting the data into training and testing sets

In [0]:
from sklearn.model_selection import train_test_split

X = pd.DataFrame(np.c_[df['LSTAT'], df['RM']], columns = ['LSTAT','RM'])
Y = df['MEDV']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, random_state = 10)

print("X_train = ", X_train.shape, "X_test = ", X_test.shape, 
      "Y_train = ", Y_train.shape, "Y_test=",   Y_test.shape)

Creating the linear model

In [0]:
lmodel = LinearRegression()
lmodel.fit(X_train, Y_train)

## Evaluating the model

In [0]:
y_train_predicted = lmodel.predict(X_train)
y_test_predicted  = lmodel.predict(X_test)

### Checking the residuals

In [0]:
plt.figure(figsize=(6,5))
plt.scatter(y_train_predicted, y_train_predicted - Y_train, c='blue', label='Training data')
plt.scatter(y_test_predicted, y_test_predicted - Y_test, c='green', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.hlines(y=0, xmin=-10, xmax=50, lw=2, color='red')
plt.xlim([-10,50])
plt.ylim([-30,20])
plt.legend(loc='upper left')

* The model does not seem to be completely wrong, as the residuals are randomly scattered around the centerline.
* Therefore, the model is unable to capture some exploratory information, as can be seen in the presence of small patterns


### Computing the MSE, RMSE, and $R^2$

* The **mean square error (MSE)** is simply the average value of the **residual sum of squares (RSS)** that we minimize to fit the linear regression model. 

$$MSE = \frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y_i})^2$$

* The **root mean square error (RMSE)** comprises the standard deviation of the residuals. It tells us how concentrated the data are around the line of the best fit.

$$RMSE = \sqrt{MSE}$$

* The coefficient of determination ($R^2$) represents the fraction of response variance that is captured by the model. It is computed as follows:

$$ R^2 = 1 - \frac{MSE}{Var(y)}$$

For the training dataset, $R^2$ is bounded between 0 and 1. Therefore, it can become negative for the test set. If $R^2 = 1$, the model fits the data perfectly, which corresponde a $MSE = 0$.



In [0]:
from sklearn.metrics import mean_squared_error, r2_score

mse_train = mean_squared_error(Y_train, y_train_predicted)
mse_test = mean_squared_error(Y_test, y_test_predicted)

print('MSE train: %.3f, test: %.3f' %(mse_train, mse_test))
print('RMSE train: %.3f, RMSE test: %.3f' % (np.sqrt(mse_train), np.sqrt(mse_test)))

print('R2 score train: %.3f, R2 score test: %.3f' %(r2_score(Y_train, y_train_predicted), 
                                                    r2_score(Y_test, y_test_predicted)))